In [1]:
import numpy as np

### 곱셈계층

In [2]:
class MulLayer:
    def __init__(self):
        self.x=None
        self.y=None

    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x*y
        return out

    def backward(self,dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx,dy

In [4]:
apple = 100
apple_num = 2
tax = 1.1 

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파 : feed-forward
apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)



220.00000000000003


In [5]:
dprice = 1
dapple_price,dtax = mul_tax_layer.backward(dprice)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple,dapple_num,dtax)


2.2 110.00000000000001 200


### 덧셈 계층

In [6]:
class AddLayer:
    def __init__(self):
        self.x=None
        self.y=None

    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x+y

        return out 
    
    def backward(self,dout):
        dx = dout * 1 
        dy = dout * 1

        return dx,dy

In [8]:
apple_num = 2
apple = 100
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_fruit_layer = AddLayer()
mul_tax_layer = MulLayer()

In [10]:
# feed-forward
apple_price = mul_apple_layer.forward(apple,apple_num)
orange_price = mul_orange_layer.forward(orange,orange_num)
all_price = add_fruit_layer.forward(apple_price,orange_price)
price = mul_tax_layer.forward(all_price,tax)

# backward
dprice = 1
dall_price,dtax = mul_tax_layer.backward(dprice)
dapple_price,dorange_price = add_fruit_layer.backward(dall_price)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)
dorange,dorange_num = mul_orange_layer.backward(dorange_price)

print(price)
print(dapple_num,dapple,dorange,dorange_num,dtax)


715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


### 5.5 활성화 함수 계층 구현하기

In [11]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self,x):
        self.mask = (x <= 0)
        out = x.copy() 
        out[self.mask] = 0

        return out
    
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout

        return dx


In [12]:
class Sigmoid:
    def __init__(self):
        self.out = None 

    def forward(self,x):
        out = 1 / 1 + np.exp(-x)
        self.out = out

        return out

    def backward(self,dout):
        dx = dout * self.out * (1-self.out)

        return dx

### Affine/Softmax 계층 구현하기

In [13]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def foward(self,x):
        self.x = x
        out = np.dot(x,self.W) + self.b

        return out 

    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)

        return dx

In [14]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self,t)
        return self.loss

    def backward(self,dout=1):
        batch_size = self.t.shape[0]

        dx = (self.y - self.t) / batch_size
        return dx

In [17]:
import sys,os
sys.path.append(os.pardir)
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [18]:
class TwoLayerNet:

    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):
        self.params={}
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params['b1'] = np.zeros_like(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b2'] = np.zeros_like(output_size )

        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(W=self.params['W1'],b=self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(W=self.params['W2'],b=self.params['b2'])
        self.layers['Relu2'] = Relu()

        self.lastLayer = SoftmaxWithLoss()

    def predict(self,x):
        for layer in self.layers:
            x = layer.forward(x)
        
        return x

    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        if t.ndim != 1 :
            np.argmax(t,axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])

        return accuracy

    def numerical_gradient(self,x,t):
        loss_W = lambda W:self.loss(x,t)

        grads={}
        grads['W1'] = numerical_gradient(loss_W,self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W,self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W,self.params['W2'])
        grads['W2'] = numerical_gradient(loss_W,self.params['b2'])

        return grads

    def gradient(self,x,t):
        self.loss(x,t)

        dout=1
        dout=1*self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = self.layer.backward(dout)

        grads={}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

In [1]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label=True)
network = TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list=[]
train_acc_list=[]
test_acc_list=[]

iter_per_epoch = max(train_size/batch_size)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.gradient(x_batch,t_batch)

    for key in ('W1','b1','W2','b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch,t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train,t_batch)
        test_acc = network.accuracy(x_test,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc,test_acc)




ModuleNotFoundError: No module named 'two_layer_net'